Note: code from this website was used (changes were made to it): \\
https://achimoraites.medium.com/lightweight-roberta-sequence-classification-fine-tuning-with-lora-using-the-hugging-face-peft-8dd9edf99d19 \\


Install necessary Libraries

In [2]:
!pip install transformers datasets evaluate accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Import required libraries

In [3]:
import torch
from torch.nn.functional import softmax
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from sklearn.metrics import f1_score, classification_report
import numpy as np
import os

# Preprocessing


Define model and tokenizer names

In [ ]:
peft_model_name = 'roberta-base-peft'
modified_base = 'roberta-base-modified'
base_model = 'roberta-base'

Load the tokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(base_model)


Encode sentences for input to the model

In [ ]:
def encode(examples):
    return tokenizer(examples['sentence'], truncation=True, padding=True)

Load data and convert labels to numeric format

In [4]:
train_data = pd.read_csv('train_en.tsv', sep='\t')
train_data['label'] = train_data['label'].apply(lambda x: 1 if x == 'SUBJ' else 0)

dev_data = pd.read_csv('dev_test_en.tsv', sep='\t')
dev_data['label'] = train_data['label'].apply(lambda x: 1 if x == 'SUBJ' else 0)

test_data = pd.read_csv('test_en_gold.tsv', sep='\t')
test_data['label'] = test_data['label'].apply(lambda x: 1 if x == 'SUBJ' else 0)

In [5]:
# Class distribution
label_distribution = train_data['label'].value_counts()
print(label_distribution)

# Percentage distribution
label_percentage = train_data['label'].value_counts(normalize=True) * 100
print(label_percentage)


label
0    532
1    298
Name: count, dtype: int64
label
0    64.096386
1    35.903614
Name: proportion, dtype: float64


Convert dataframes to HuggingFace datasets

In [ ]:
train_dataset = Dataset.from_pandas(train_data)
dev_dataset = Dataset.from_pandas(dev_data)
test_dataset = Dataset.from_pandas(test_data)

train_dataset = train_dataset.map(encode, batched=True, remove_columns=["solved_conflict", "sentence_id"])
dev_dataset = dev_dataset.map(encode, batched=True, remove_columns=["sentence_id"])
test_dataset = test_dataset.map(encode, batched=True, remove_columns=["sentence_id"])

Map:   0%|          | 0/830 [00:00<?, ? examples/s]

Map:   0%|          | 0/243 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

Create label mappings

In [ ]:
# Create an id2label mapping
# We will need this for our classifier.
id2label = {0: "OBJ", 1: "SUBJ"}
label2id = {"OBJ": 0, "SUBJ": 1}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Model 0: Baseline RoBERTa Model

Load pre-trained RoBERTa model for classification

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluate baseline model

In [ ]:
from tqdm import tqdm

def evaluate_model(inference_model, dataset, data_collator):

    eval_dataloader = DataLoader(
        dataset,
        batch_size=8,
        collate_fn=data_collator  # Handles padding
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inference_model.to(device)
    inference_model.eval()

    all_predictions = []
    all_references = []

    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        references = batch["labels"]

        all_predictions.extend(predictions.cpu().numpy())
        all_references.extend(references.cpu().numpy())

    report = classification_report(all_references, all_predictions, digits=4)
    print("Classification Report:\n")
    print(report)

# Evaluate the non fine-tuned model
test_dataset_torch = test_dataset
test_dataset_torch.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
evaluate_model(model, test_dataset_torch, data_collator)

100%|██████████| 61/61 [00:03<00:00, 15.98it/s]

Classification Report:

              precision    recall  f1-score   support

           0     0.7479    1.0000    0.8558       362
           1     0.0000    0.0000    0.0000       122

    accuracy                         0.7479       484
   macro avg     0.3740    0.5000    0.4279       484
weighted avg     0.5594    0.7479    0.6401       484




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Model 1: Fine-tuned RoBERTa


In [ ]:
model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

Training Model

In [ ]:
os.environ["WANDB_MODE"] = "disabled"

# Set training hyperparameters
learning_rate = 3e-5
epochs = 25
warmup_steps = 500

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    logging_dir='./logs',
    learning_rate=learning_rate,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=epochs,
    warmup_steps=warmup_steps,
    weight_decay=0.01
)

# Initialize Trainer with model, training arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start the training process
trainer.train()

<ipython-input-14-964f6982f067>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.045800
1000,0.025800


TrainOutput(global_step=1300, training_loss=0.027536203616227096, metrics={'train_runtime': 571.537, 'train_samples_per_second': 36.306, 'train_steps_per_second': 2.275, 'total_flos': 1535499674640000.0, 'train_loss': 0.027536203616227096, 'epoch': 25.0})

Evaluate Model on Test Dataset

In [ ]:
# Evaluate the model on the test dataset
test_results = trainer.predict(test_dataset)
logits = test_results.predictions

# Convert logits to predicted class indices
predicted_labels = np.argmax(logits, axis=-1)

# predicted_labels = [id2label[idx] for idx in predicted_class_indices]

actual_labels = test_dataset["label"]

f1 = f1_score(actual_labels, predicted_labels, average="weighted")

# Print results
print("F1-score:", f1)
print("Classification Report:")
print(classification_report(actual_labels, predicted_labels, target_names=["Objective", "Subjective"]))

F1-score: 0.8096142953185782
Classification Report:
              precision    recall  f1-score   support

   Objective       0.83      0.96      0.89       362
  Subjective       0.79      0.43      0.56       122

    accuracy                           0.83       484
   macro avg       0.81      0.70      0.73       484
weighted avg       0.82      0.83      0.81       484



# Model 2: LoRA Fine-Tuning leveraging the PEFT (Parameter-Efficient Fine-Tuning) library

In [ ]:
# Configure the LoRA settings for parameter-efficient fine-tuning
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)

# Create a PEFT model using LoRA configuration
peft_model = get_peft_model(model, peft_config)

In [ ]:
peft_model.print_trainable_parameters()

trainable params: 887,042 || all params: 125,534,212 || trainable%: 0.7066


Train the Model

In [ ]:
# Set hyperparameters for training
learning_rate = 3e-5
epochs = 25
warmup_steps = 500

# Define training arguments for LoRA
training_args = TrainingArguments(
    output_dir='./lora_results',
    logging_dir='./lora_logs',
    learning_rate=learning_rate,  # You can experiment with slightly higher learning rates for LoRA (e.g., 2e-4)
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=epochs,
    warmup_steps=warmup_steps,
    weight_decay=0.01,
)

# Initialize the Trainer with the PEFT model, datasets, and training arguments
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start the training process
trainer.train()

<ipython-input-20-1e3aa1c9e233>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.652200
1000,0.392100


TrainOutput(global_step=1300, training_loss=0.47892373891977164, metrics={'train_runtime': 371.8962, 'train_samples_per_second': 55.795, 'train_steps_per_second': 3.496, 'total_flos': 1551402563616000.0, 'train_loss': 0.47892373891977164, 'epoch': 25.0})

Evaluate the model on the test dataset

In [ ]:
test_results = trainer.predict(test_dataset)
logits = test_results.predictions

# Convert logits to predicted class indices
predicted_labels = np.argmax(logits, axis=-1)

# predicted_labels = [id2label[idx] for idx in predicted_class_indices]

actual_labels = test_dataset["label"]

f1 = f1_score(actual_labels, predicted_labels, average="weighted")

# Print results
print("F1-score:", f1)
print("Classification Report:")
print(classification_report(actual_labels, predicted_labels, target_names=["Objective", "Subjective"]))

F1-score: 0.7830971413087657
Classification Report:
              precision    recall  f1-score   support

   Objective       0.81      0.97      0.89       362
  Subjective       0.79      0.34      0.48       122

    accuracy                           0.81       484
   macro avg       0.80      0.66      0.68       484
weighted avg       0.81      0.81      0.78       484

